# Revised case normalization for Hirslanden St. Anna Luzern 2018

Notes:

- Inserted 51 cases into the DB


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Klinik St. Anna in Luzern', '2018', 'Änderungen_ST. Anna_2018')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Klinik St. Anna in Luzern', year='2018', sheet='Änderungen_ST. Anna_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 09:09:56.590 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 51 cases for Hirslanden Klinik St. Anna in Luzern 2018
2022-11-21 09:09:56.596 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
tranche                   string
datum                     string
case_id                   object
patient_id                object
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-änderung mögl

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0009358410,EB395DA315D5B285,W,79,9,9358410,M169,M169,M200,3,I46C,[D62],[D648],[],[]
1,0009364697,6586D083924C0E99,W,68,16,9364697,M4806,M4806,M200,3,I09C,[E1191],[E1190],[],[]
2,0009366989,9FBF3AE4CEA4A950,M,27,16,9366989,M511,M511,M200,3,I10C,[B956],[],[],[]
3,0009371704,FDE409238A9BEC4F,M,65,13,9371704,I5001,I5001,M100,3,F49C,[N183],[],[],[]
4,0009375929,036F0863E37E3493,M,74,18,9375929,M4806,M4806,M200,3,I27C,[T846],[T844],[],[]
5,0009377457,B172FAC0D995D8A0,M,77,5,9377457,I5000,I5000,M100,3,F62C,"[K746, E870]",[],[],[]
6,0009381325,E3D0603E52140F32,W,55,15,9381325,J4410,J4410,M100,3,E65B,[J9610],[],[],[]
7,0009382082,4ED15568FB7BDE9E,M,82,8,9382082,J100,J100,M100,3,E77E,[E870],[],[],[]
8,0009384055,44197C8BC8569771,W,87,14,9384055,S8180,S8180,M200,3,X06B,[E871],[],[],[]
9,0009391002,8BD23E59AF760596,M,64,14,9391002,J128,J128,M100,3,E77E,[C9000],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-21 09:10:18.558 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 12985 rows from the DB, for the hospital 'Hirslanden Klinik St. Anna in Luzern' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,265595,0009384055,S8180,"[X599, L908, E440, F328, I1090, ...]",867A9E:L:20180219,"[9938::20180219, 890A32::20180223]",W,87,0,0,14,0,01,2018-02-19,0,00,2018-03-05
1,266310,0009394697,I723,"[T823, Y828, I7021, I7020, I2519, ...]",3846:R:20180418,"[394920:R:20180418, 99B711::20180418]",M,66,0,0,16,0,01,2018-04-17,0,00,2018-05-03
2,266360,0009443438,J869,"[J13, I480, J4489, J9600]",345215:R:20180817,"[332411:B:20180816, 3491:R:20180816, 992217::20180816, 345910:R:20180817, 3406:R:20180817, ...]",M,52,0,0,5,0,11,2018-08-16,0,06,2018-08-21
3,266870,0009403476,T821,"[Y828, I420, I489, Z921, I1090, ...]",377B11:R:20180420,"[377A11:R:20180420, 377B13::20180420, 377A13::20180420, 377531:L:20180420]",M,76,0,0,4,0,01,2018-04-19,0,00,2018-04-23
4,267730,0009419360,C184,"[C772, K567]",457711::20180607,"[457511::20180607, 403X29::20180607, 009A13::20180607, 4664::20180607, 99B711::20180607]",M,27,0,0,7,0,01,2018-06-06,0,00,2018-06-13
5,267767,0009447736,C862,"[K900, K631, K660, K210, E441, ...]",4562::20180905,"[4525::20180828, 4516::20180828, 890A32::20180903, 544X12::20180905, 99B811::20180905]",M,64,0,0,18,0,01,2018-08-28,0,00,2018-09-15
6,268109,9475859,C830,"[M313, J9580, M500, G992, J4482, ...]",3327:L:20181123,"[332410::20181123, 3404:L:20181123, 889714::20181125, 889110::20181125]",M,58,0,0,6,0,01,2018-11-22,0,00,2018-11-28
7,268201,0009446567,J208,"[B965, J459, G2581, G4731, Z436, ...]",992218::20180824,[891711::20180903],W,90,0,0,11,0,01,2018-08-24,0,00,2018-09-04
8,268233,0009358410,M169,"[I2513, Z950, Z955, I420, E210, ...]",815111:L:20180201,"[009940::20180201, 009960::20180201, 9900::20180201, 948X40::20180202, 990410::20180207]",W,79,0,0,9,0,01,2018-02-01,0,00,2018-02-10
9,268308,0009366989,M511,"[G9780, T814, G9788, T813, G474, ...]",862A2E::20171225,"[035913::20171225, 030934::20171225, 848199::20171225, 99221A::20171225, 99B812::20171225, ...]",M,27,0,0,16,0,01,2017-12-25,0,00,2018-01-10


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:10:19.658 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 51 cases ...
2022-11-21 09:10:20.772 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 51 cases into: 51 revisions, 455 diagnoses rows, 265 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,265595,X06A,2.83,2.83,4,2022-12-31
1,266310,F34C,3.88,3.88,4,2022-12-31
2,266360,E05A,3.39,1.63,4,2022-12-31
3,266870,F18A,2.44,2.44,4,2022-12-31
4,267730,G18A,2.96,2.96,4,2022-12-31
5,267767,R01B,4.20,4.20,4,2022-12-31
6,268109,R61B,1.77,1.77,4,2022-12-31
7,268201,E69A,0.93,0.93,4,2022-12-31
8,268233,I46A,2.61,2.61,4,2022-12-31
9,268308,I10A,2.66,2.66,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:10:20.793 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 51 cases into the 'Revisions' table ...
2022-11-21 09:10:22.097 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 51 cases into the 'Revisions' table
2022-11-21 09:10:22.099 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 455 rows into the 'Diagnoses' table ...
2022-11-21 09:10:24.359 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 455 rows into the 'Diagnoses' table
2022-11-21 09:10:24.362 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 265 rows into the 'Procedures' table ...
2022-11-21 09:10:26.240 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 265 rows into the 'Procedures' table
